# Report for May 4

In [1]:
import utils
import shared_utils
from dla_utils import _dla_utils as dla_utils
from shared_utils import styleguide, altair_utils, portfolio_utils

from siuba import *
import pandas as pd

from IPython.display import display, Markdown, HTML

import altair as alt

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl

from calitp import query_sql, magics


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
### Using BBB as agency

In [3]:
## alternatively 
may4 = query_sql(f'''
SELECT * FROM `cal-itp-data-infra.views.gtfs_rt_vs_schedule_trips_may4_sample`
''')

In [4]:
may4['service_date'] = pd.to_datetime(may4['service_date'])
may4['weekday'] = pd.Series(may4.service_date).dt.day_name()    
may4['month'] =  pd.Series(may4.service_date).dt.month_name()

In [5]:
itpid_district = portfolio_utils.add_caltrans_district()
may4 = pd.merge(may4, itpid_district, on='calitp_itp_id', how='left')

In [6]:
may4 = utils.get_correct_url(may4)

In [7]:
may4.sample(5)

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
1565,76,Commute.org Shuttles,0,12161,OTP Shuttle,2022-05-04,2022-01-24,2022-09-19,8,0,0.00,Wednesday,May,04 - Oakland
864,142,Orange County Transportation Authority,0,79,79,2022-05-04,2021-04-16,2099-01-01,54,0,0.00,Wednesday,May,12 - Irvine
3147,200,Livermore Amador Valley Transit Authority,0,WH:53,53,2022-05-04,2021-07-31,2099-01-01,8,6,0.75,Wednesday,May,None
555,174,LA Go Bus,0,13050,None,2022-05-04,2021-04-15,2099-01-01,23,0,0.00,Wednesday,May,NaN
1072,314,Sonoma County Transit,1,32,32,2022-05-04,2021-12-29,2099-01-01,22,0,0.00,Wednesday,May,04 - Oakland


In [8]:
dist_avg = (utils.get_agg_pct(may4, groupings = ['caltrans_district'], sum_sched = 'num_sched', sum_vp = 'num_vp'))
dist_avg['District Average'] = dist_avg['avg'].astype(float).map("{:.2%}".format)

In [9]:
display(HTML(dla_utils.pretify_tables((dist_avg>>select(_.caltrans_district, _['District Average'])>>filter(_.caltrans_district.notnull())>>arrange(_.caltrans_district)))))

Caltrans District,District Average
01 - Eureka,25.59%
02 - Redding,29.60%
03 - Marysville,4.26%
04 - Oakland,64.13%
05 - San Luis Obispo,26.97%
06 - Fresno,39.14%
07 - Los Angeles,68.81%
08 - San Bernardino,52.52%
09 - Bishop,62.90%
10 - Stockton,68.23%


In [10]:
may4.sample()

,calitp_itp_id,agency_name,calitp_url_number,route_id,route_short_name,service_date,calitp_extracted_at,calitp_deleted_at,num_sched,num_vp,pct_w_vp,weekday,month,caltrans_district
1655,278,MTS,0,1,1,2022-05-04,2022-01-30,2022-05-19,126,122,0.97,Wednesday,May,11 - San Diego


In [11]:
ag_avg = (utils.get_agg_pct(may4,
                            groupings = ['agency_name','caltrans_district'],
                            sum_sched = 'num_sched',
                            sum_vp = 'num_vp'))

In [12]:
ag_avg

,agency_name,caltrans_district,num_sched,num_vp,avg
0,County Connection,None,NaN,NaN,NaN
1,RTA - SCT - Paso,05 - San Luis Obispo,190.00,0.00,0.00
2,Ventura County Transportation Commission,07 - Los Angeles,214.00,0.00,0.00
3,Valley Express,07 - Los Angeles,35.00,0.00,0.00
4,Tri Delta Transit,04 - Oakland,369.00,358.00,0.97
...,...,...,...,...,...
219,Marin Transit,None,NaN,NaN,NaN
220,Glenn Transit Service,03 - Marysville,10.00,0.00,0.00
221,Yosemite Area Regional Transportation System,10 - Stockton,12.00,0.00,0.00
222,TRACER,10 - Stockton,69.00,0.00,0.00


In [13]:
#trying using utils bar chart
display(HTML('<strong>Agency Average Scheduled & RT Vehicle Position Data</strong>'))
scatter = (utils.bar_chart_over_time(ag_avg,
                          x_col = 'num_sched', 
                          y_col = 'num_vp', 
                          color_col = 'caltrans_district', 
                          yaxis_format = '',
                          sort = 'x', 
                          title_txt = '')).mark_circle(size=60).interactive()
(utils.add_tooltip(scatter, 'agency_name', 'avg', 'caltrans_district')).properties(width=700)

alt.Chart(...)